In [1]:
import pandas as pd
import numpy as np
import re
import string
from textblob import TextBlob
from nltk.stem.snowball import SnowballStemmer
from nltk.tokenize import word_tokenize 
from nltk.corpus import stopwords 


from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier

In [2]:
new_train = pd.read_csv("train_limpio_con_Tf-Idf.csv",encoding = "ISO-8859-1")
new_train.head(5)

,aba,abandon,abc,abcnew,abl,ablaz,absolut,abstorm,abus,access,...,youth,youtub,youv,yr,yrs,yyc,zionist,zombi,zone,target
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1


In [3]:
new_test = pd.read_csv("test_limpio_con_Tf-Idf.csv",encoding = "ISO-8859-1")
new_test.head()

,aba,abandon,abc,abcnew,abl,ablaz,absolut,abstorm,abus,access,...,your,youth,youtub,youv,yr,yrs,yyc,zionist,zombi,zone
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [4]:
new_train = new_train.drop(columns = ['id'])

In [5]:
train_limpio_true = new_train.loc[new_train["target"] == 1]
train_limpio_false = new_train.loc[new_train["target"] == 0]
print("Tenemos {cant_verdaderos: .2f} verdaderos y {cant_falsos: .2f} falsos".format(cant_verdaderos=len(train_limpio_true), cant_falsos=len(train_limpio_false)))

Tenemos  3271.00 verdaderos y  4342.00 falsos


In [6]:
train = pd.concat([train_limpio_true.iloc[:3000], train_limpio_false.iloc[:3850]])
val = pd.concat([train_limpio_true.iloc[3000:], train_limpio_false.iloc[3850:]])


In [7]:
X_train = train.drop(columns=['target'])
Y_train = train['target']
X_val = val.drop(columns=['target'])
Y_val = val['target']

In [12]:

model1 = Sequential()
model1.add(layers.Embedding(2392,32))
model1.add(layers.GlobalMaxPool1D())
model1.add(layers.Dense(10, activation='relu'))
model1.add(layers.Dense(1, activation='sigmoid'))

model1.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['acc'])
model1.summary()



Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 32)          76544     
_________________________________________________________________
global_max_pooling1d (Global (None, 32)                0         
_________________________________________________________________
dense_4 (Dense)              (None, 10)                330       
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 11        
Total params: 76,885
Trainable params: 76,885
Non-trainable params: 0
_________________________________________________________________


In [13]:
first_history = model1.fit(X_train.to_numpy(), Y_train.to_numpy(),
                    epochs=2,
                    verbose=2,
                    validation_data=(X_val.to_numpy(), Y_val.to_numpy()),
                    batch_size=10)

Epoch 1/2
685/685 - 1s - loss: 0.6858 - acc: 0.5620 - val_loss: 0.6627 - val_acc: 0.6448
Epoch 2/2
685/685 - 1s - loss: 0.6844 - acc: 0.5620 - val_loss: 0.6621 - val_acc: 0.6448


In [53]:


input_dim = 2392  # Number of features

model2 = Sequential()
model2.add(layers.Dense(7, input_dim=input_dim, activation='relu'))
model2.add(layers.Dense(1, activation='sigmoid'))

model2.compile(loss='binary_crossentropy', 
              optimizer='adam', 
              metrics=['acc'])
model2.summary()



Model: "sequential_23"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_46 (Dense)             (None, 7)                 16751     
_________________________________________________________________
dense_47 (Dense)             (None, 1)                 8         
Total params: 16,759
Trainable params: 16,759
Non-trainable params: 0
_________________________________________________________________


In [54]:
first_history = model2.fit(X_train.to_numpy(), Y_train.to_numpy(),
                    epochs=2,
                    verbose=2,
                    validation_data=(X_val.to_numpy(), Y_val.to_numpy()),
                    batch_size=10)

Epoch 1/2
685/685 - 0s - loss: 0.6138 - acc: 0.7333 - val_loss: 0.5216 - val_acc: 0.8126
Epoch 2/2
685/685 - 0s - loss: 0.4386 - acc: 0.8231 - val_loss: 0.5009 - val_acc: 0.7575


In [55]:
loss, accuracy = model2.evaluate(X_val.to_numpy(), Y_val.to_numpy(), verbose=False)
print("Testing Accuracy:  {:.4f}".format(accuracy))

Testing Accuracy:  0.7575


In [56]:
firstPredictions = list(map(lambda x: x[0], model2.predict_classes(new_test.drop(columns=['id']).to_numpy())))
len(firstPredictions)

Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).


7613

In [57]:
firstPredictionsDF = pd.DataFrame(data={'id': new_test['id'], 'target': firstPredictions})
firstPredictionsDF.head(10)

,id,target
0,0.0,1
1,0.0,1
2,0.0,1
3,0.0,1
4,0.0,0
5,0.0,1
6,0.0,1
7,0.0,0
8,0.0,1
9,0.0,1


In [58]:
firstPredictionsDF.to_csv('Tfidf_firstPredictions.csv', index=False)

QUIERO REDUCIR LAS DIMENSIONES PARA PODER APLICAR UN MODELO MAS COMPLEJO SIN QUE OVERFITTEE

In [60]:
new_train.head()

,aba,abandon,abc,abcnew,abl,ablaz,absolut,abstorm,abus,access,...,youth,youtub,youv,yr,yrs,yyc,zionist,zombi,zone,target
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1


In [71]:
lista_columnas = new_train.columns.to_list()


In [74]:
# How many elements each  
# list should have  
n = 8
   
# using list comprehension  
lista_dividida = [lista_columnas[i:i + n] for i in range(0, len(lista_columnas), n)]   

In [75]:
lista_dividida.remove(['target'])